In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
games_df = pd.read_csv("data/games.csv")
games_df.sample(10)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
63172,2072950,Heavy Works,"Sep 3, 2022",0 - 20000,0,0,10.79,0,Heavy Works puts you in an openworld town wher...,['English'],...,0,0,0,TSRS,TSRS,Single-player,"Casual,Indie,Simulation,Early Access",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
61087,755590,Battle High 2 A+,"Dec 14, 2017",0 - 20000,0,0,1.99,0,The fast-paced fighting action of the Battle H...,['English'],...,0,0,0,"Mattrified Games, LLC,Point5Projects","Mattrified Games, LLC","Single-player,Multi-player,PvP,Shared/Split Sc...","Action,Indie","Action,Indie,Fighting,2D Fighter",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
67579,2216740,Zombie Land - Survival Playtest,"Dec 17, 2022",0 - 0,0,0,0.00,0,NaN,[],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61488,1808260,Cute Waifu,"May 17, 2022",0 - 20000,0,0,0.99,0,Gameplay: Move All Parts to Form the Complete ...,['English'],...,0,0,0,Reddiamondgames,Reddiamondgames,Single-player,"Casual,Indie",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
46447,35140,Batman: Arkham Asylum Game of the Year Edition,"Mar 26, 2010",2000000 - 5000000,455,0,19.99,0,Critically acclaimed Batman: Arkham Asylum ret...,"['English', 'French', 'German', 'Italian', 'Sp...",...,0,381,0,Rocksteady Studios,Warner Bros. Interactive Entertainment,"Single-player,Steam Achievements,Steam Trading...","Action,Adventure","Action,Stealth,Third Person,Superhero,Adventur...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/5...
62723,2123730,Championship Wrestling Promoter Playtest,"Aug 22, 2022",0 - 0,0,0,0.00,0,NaN,[],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66309,2202720,One Man's War,"Nov 15, 2022",0 - 20000,0,0,0.69,0,Chiyou fought against the Yellow Emperor nine ...,"['English', 'Japanese', 'Simplified Chinese']",...,0,0,0,Dawn Studio,Dawn Studio,Single-player,"Action,Adventure,Indie",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
55745,1685520,Hyper Treasure - Macaronic Labyrinth,"Oct 29, 2021",0 - 20000,0,0,1.99,0,"Treasure hunters, Noracam and Macarsha were re...","['English', 'French', 'Spanish - Latin America...",...,0,0,0,Rusimitu Games,Rusimitu Games,"Single-player,Steam Achievements,Full controll...","Adventure,Casual,Indie","Point & Click,Casual,FMV,Adventure,Interactive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2247,1593080,Romance of Raskya,"Jun 2, 2021",0 - 20000,0,0,6.99,0,The city of Dagara is burning. For years the r...,['English'],...,0,0,0,"Basileus Games, LLC","Basileus Games, LLC","Single-player,Steam Achievements",Indie,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
7068,819560,TurbOT Racing,"Apr 13, 2018",0 - 20000,0,0,0.99,1,from the Developer... Welcome to Turbot Racing...,"['English', 'Portuguese - Brazil', 'Simplified...",...,0,0,0,creasso,creasso,"Single-player,Steam Achievements,Full controll...","Action,Indie,Racing","Action,Indie,Racing,Sci-fi,Robots,Sports,Arcade",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [3]:
games_df['Tags'].str.split(',')

0                         [Indie, Casual, Sports, Bowling]
1        [Indie, Action, Pixel Graphics, 2D, Retro, Arc...
2                                                      NaN
3        [2D Platformer, Atmospheric, Surreal, Mystery,...
4        [Indie, Adventure, Nudity, Violent, Sexual Con...
                               ...                        
74684                                                  NaN
74685                                                  NaN
74686    [Action, Strategy, Fantasy, Fighting, Retro, T...
74687    [Violent, Gore, Action, Indie, Adventure, VR, ...
74688                                                  NaN
Name: Tags, Length: 74689, dtype: object

In [4]:
games_df['Tags'] = games_df['Tags'].str.split(',')
games_df['Categories'] = games_df['Categories'].str.split(',')
games_df['Genres'] = games_df['Genres'].str.split(',')

# games_df['Tags'] = games_df['Tags'].apply(lambda x: [tag.lower() for tag in x] if isinstance(x, list) else [])
# games_df['Categories'] = games_df['Categories'].apply(lambda x: [category.lower() for category in x] if isinstance(x, list) else [])
# games_df['Genres'] = games_df['Genres'].apply(lambda x: [genre.lower() for genre in x] if isinstance(x, list) else [])
games_df[['Genres', 'Categories', 'Tags']]


,Genres,Categories,Tags
0,"[Casual, Indie, Sports]","[Single-player, Multi-player, Steam Achievemen...","[Indie, Casual, Sports, Bowling]"
1,"[Action, Indie]","[Single-player, Steam Achievements, Full contr...","[Indie, Action, Pixel Graphics, 2D, Retro, Arc..."
2,"[Action, Adventure, Indie, Strategy]",[Single-player],NaN
3,"[Adventure, Casual, Indie]","[Single-player, Full controller support]","[2D Platformer, Atmospheric, Surreal, Mystery,..."
4,"[Adventure, Indie]","[Single-player, Steam Achievements]","[Indie, Adventure, Nudity, Violent, Sexual Con..."
...,...,...,...
74684,[Adventure],"[Single-player, Multi-player, Co-op, Online Co...",NaN
74685,NaN,NaN,NaN
74686,"[Action, Strategy]",[Single-player],"[Action, Strategy, Fantasy, Fighting, Retro, T..."
74687,"[Action, Adventure, Indie, RPG]","[Single-player, Steam Achievements, Tracked Co...","[Violent, Gore, Action, Indie, Adventure, VR, ..."


In [5]:
games_df['combined_features'] = games_df['Tags'] + games_df['Categories'] + games_df['Genres']
games_df['combined_features'] 

0        [Indie, Casual, Sports, Bowling, Single-player...
1        [Indie, Action, Pixel Graphics, 2D, Retro, Arc...
2                                                      NaN
3        [2D Platformer, Atmospheric, Surreal, Mystery,...
4        [Indie, Adventure, Nudity, Violent, Sexual Con...
                               ...                        
74684                                                  NaN
74685                                                  NaN
74686    [Action, Strategy, Fantasy, Fighting, Retro, T...
74687    [Violent, Gore, Action, Indie, Adventure, VR, ...
74688                                                  NaN
Name: combined_features, Length: 74689, dtype: object

In [16]:
# Compute the TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(games_df['combined_features'].astype(str))

# Compute the cosine similarity matrix
cosine_similarities_TFIDF = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [6]:
# Compute the count matrix using CountVectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(games_df['combined_features'].astype(str))

# Compute the cosine similarity matrix using the count matrix
cosine_similarities_CV = cosine_similarity(count_matrix, count_matrix)

In [9]:
# Define a function to get recommendations based on a game's index
def get_recommendations(game_index, cosine_similarities, df=games_df):
    # Get the similarity scores of the game with other games
    similarity_scores = list(enumerate(cosine_similarities[game_index]))

    # Sort the games based on similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 similar games
    top_10_indices = [i[0] for i in similarity_scores[1:11]]

    # Return the top 10 similar games
    return df.iloc[top_10_indices]['Name']

In [21]:
# Get recommendations for a game at index 0
recommendations = get_recommendations(7357)
print(recommendations)

61394           Earth 2150 Trilogy
40986                Bionic Attack
43433            Trapper's Delight
47920                    MegaGlest
5179     World War III: Black Gold
49065                Cyber Complex
46356         Planets Under Attack
8051                     Orch Star
10151                 8-Bit Hordes
6070                    Earth 2140
Name: Name, dtype: object


In [10]:
# Get recommendations for a game at index 0
recommendations = get_recommendations(7357, cosine_similarities=cosine_similarities_CV)
print(recommendations)

43433                     Trapper's Delight
30162                              Etherium
5721                     Crush Your Enemies
59856                           BallisticNG
5503     Age of Empires: Definitive Edition
61394                    Earth 2150 Trilogy
37095                          Nuclear Dawn
46356                  Planets Under Attack
20361                       1775: Rebellion
6070                             Earth 2140
Name: Name, dtype: object
